# Laboratorio 6 - Análisis de Redes Sociales

#### Diego García 22404
#### Andrés Ortega 22305
#### Esteban Zambrano 22119

**Link del repositorio:**

https://github.com/DiegoGarV/lab6-DS

*PROBLEMA 2 - BERNARDO ARÉVALO*

## Limpieza de datos

In [ ]:
import os, json, re
from typing import Any, Dict, List
import pandas as pd

# Carpeta de datos / salida
os.makedirs("data", exist_ok=True)
INPUT_TXT = "../data/tioberny.txt" 

OUTPUT_CSV = "../data/converted_data.csv"
print("Leyendo de:", INPUT_TXT)


Leyendo de: ../data/tioberny.txt


Los json con los datos incluyen una gran cantidad de metadata, de la cuál no toda es necesaria. Tomando en cuenta los datos que se iban a analizar, se dicidió solo tomar en cuenta el username, la información del tweet, los hashtags y las menciones. Además, si en un tweet se cita a otro, se tomó también como un dato importante, ya que crea una interacción. Este se va tratar por aparte de las menciones. Con esto se obtendrá el csv inicial para el laboratorio.

In [4]:
from typing import Any, Dict, List

# Detecta codificación
def sniff_encoding(path: str) -> str:
    with open(path, "rb") as f:
        raw = f.read(4)
    if raw.startswith(b"\xff\xfe\x00\x00") or raw.startswith(b"\x00\x00\xfe\xff"):
        return "utf-32"
    if raw.startswith(b"\xff\xfe") or raw.startswith(b"\xfe\xff"):
        return "utf-16"
    if raw.startswith(b"\xef\xbb\xbf"):
        return "utf-8-sig"
    return "utf-8"


"""
1) Detecta codificación.
2) Intenta parsear TODO como JSON (lista u objeto).
3) Si falla, interpreta como JSONL (una línea = un objeto JSON).
Ignora líneas vacías o malformadas en modo JSONL.
"""
def load_tweets_from_txt(path: str) -> List[Dict[str, Any]]:
    enc = sniff_encoding(path)
    print(f"[load] Codificación detectada: {enc}")

    # Caso 1: archivo entero como JSON
    with open(path, "r", encoding=enc) as f:
        content = f.read().strip()

    try:
        parsed = json.loads(content)
        if isinstance(parsed, list):
            return parsed
        elif isinstance(parsed, dict):
            return [parsed]
    except Exception:
        pass  # seguimos con JSONL

    # Caso 2: JSON Lines (una línea por objeto)
    tweets: List[Dict[str, Any]] = []
    bad, total = 0, 0
    with open(path, "r", encoding=enc) as f:
        for line in f:
            line = line.strip()
            if not line or not line.startswith("{"):
                continue
            total += 1
            try:
                obj = json.loads(line)
                if isinstance(obj, dict):
                    tweets.append(obj)
            except Exception:
                bad += 1
                continue

    print(f"[load] Registros leídos: {len(tweets)} | líneas con error: {bad}/{total}")
    return tweets

# Ejecutar carga
tweets_raw = load_tweets_from_txt(INPUT_TXT)
len(tweets_raw)


[load] Codificación detectada: utf-16
[load] Registros leídos: 5019 | líneas con error: 0/5019


5019

In [ ]:
def get_username(obj: Dict[str, Any]) -> str:
    u = obj.get("user") or {}
    return (u.get("username") or "").strip() if isinstance(u, dict) else ""

def get_raw_content(obj: Dict[str, Any]) -> str:
    return (obj.get("rawContent") or "").strip()

def get_hashtags(obj: Dict[str, Any]) -> List[str]:
    hs = obj.get("hashtags", [])
    # Pueden venir como lista de strings o lista de objetos
    out = []
    if isinstance(hs, list):
        for h in hs:
            if isinstance(h, str):
                out.append(h.strip())
            elif isinstance(h, dict):
                # por si viniera como {"text": "algo"}
                txt = h.get("text") or h.get("tag") or ""
                if txt:
                    out.append(str(txt).strip())
    return [h for h in out if h]

def get_mentions(obj: Dict[str, Any]) -> List[str]:
    ms = obj.get("mentionedUsers", [])
    out = []
    if isinstance(ms, list):
        for m in ms:
            if isinstance(m, dict):
                u = m.get("username")
                if u:
                    out.append(str(u).strip())
            elif isinstance(m, str):
                out.append(m.strip())
    return [m for m in out if m]

def get_quoted(obj: Dict[str, Any]) -> Dict[str, Any]:
    qt = obj.get("quotedTweet")
    return qt if isinstance(qt, dict) else {}

# Extrae el username del tweet citado (si existe)
def get_from_user_for_row(obj: Dict[str, Any]) -> str:
    qt = get_quoted(obj)
    return get_username(qt) if qt else ""


In [ ]:
def row_from_tweet(obj: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "Username": get_username(obj),
        "rawTweet": get_raw_content(obj),
        "hashtags": ", ".join(get_hashtags(obj)),     # en CSV quedará como texto separado por coma
        "mentions": ", ".join(get_mentions(obj)),     # idem
        "fromUser": get_from_user_for_row(obj),       # usuario del tweet citado por ESTA FILA (si existe)
    }


"""
Agrega una fila por 'obj' y luego recursivamente por cada quotedTweet anidado.
Usa seen_ids para evitar ciclos/duplicados si se repiten objetos por ID.
"""
def flatten_with_quotes(obj: Dict[str, Any], rows: List[Dict[str, Any]], seen_ids: set):
    # Intentar usar 'id' para deduplicación (si existe)
    obj_id = obj.get("id") or obj.get("id_str")
    if obj_id is not None:
        if obj_id in seen_ids:
            return
        seen_ids.add(obj_id)

    # Fila del tweet actual
    rows.append(row_from_tweet(obj))

    # Si tiene quotedTweet, aplanar recursivamente
    qt = get_quoted(obj)
    if qt:
        flatten_with_quotes(qt, rows, seen_ids)

def build_flat_table(objs: List[Dict[str, Any]]) -> pd.DataFrame:
    rows: List[Dict[str, Any]] = []
    seen_ids: set = set()
    for obj in objs:
        if isinstance(obj, dict):
            flatten_with_quotes(obj, rows, seen_ids)
    # Orden de columnas exactamente como pediste
    cols = ["Username", "rawTweet", "hashtags", "mentions", "fromUser"]
    df = pd.DataFrame(rows, columns=cols)
    # Limpieza básica de espacios
    for c in cols:
        df[c] = df[c].fillna("").astype(str).str.strip()
    return df

df_converted = build_flat_table(tweets_raw)
print(df_converted.shape)
df_converted.head(11)


(4944, 5)


,Username,rawTweet,hashtags,mentions,fromUser
0,La_ReVoluZzion,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",,"usembassyguate, 48CantonesToto, USAIDGuate, UE...",XelaNewsGt
1,XelaNewsGt,#URGENTE Lo que los medios #faferos no informa...,"URGENTE, faferos, BernardoArévalo, NebajQuiché...",,
2,M24095273,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,,"IvanDuque, BArevalodeLeon",
3,carlosalbesc,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,,"IvanDuque, BArevalodeLeon",
4,Brenda_AGN,El presidente @BArevalodeLeon y la vicepreside...,,"BArevalodeLeon, KarinHerreraVP, AGN_noticias, ...",
5,Roberto28338166,@BArevalodeLeon El muy hijo de puta inyectó co...,,BArevalodeLeon,
6,LuisEnr36669555,@EmisorasUnidas @BArevalodeLeon Y que de bueno...,,"EmisorasUnidas, BArevalodeLeon",
7,keratox1,@IvanDuque @BArevalodeLeon Productiva es que g...,,"IvanDuque, BArevalodeLeon",
8,Radio_TGW,"@KarinHerreraVP El presidente de la República,...",,"KarinHerreraVP, BArevalodeLeon",
9,Corleone_62,"@mys_servicios @BArevalodeLeon No tampoco, ese...",,"mys_servicios, BArevalodeLeon",


In [9]:
df_converted.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✓ Archivo guardado en: {OUTPUT_CSV}")

✓ Archivo guardado en: ../data/converted_data.csv


## Preprocesamiento de los datos

In [12]:
import os
import pandas as pd

INPUT_CSV = "../data/converted_data.csv"

df = pd.read_csv(
    INPUT_CSV,
    dtype={"Username": str, "rawTweet": str, "hashtags": str, "mentions": str, "fromUser": str}
).fillna("")

df.info()
display(df.head())
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4944 entries, 0 to 4943
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Username  4944 non-null   object
 1   rawTweet  4944 non-null   object
 2   hashtags  4944 non-null   object
 3   mentions  4944 non-null   object
 4   fromUser  4944 non-null   object
dtypes: object(5)
memory usage: 193.2+ KB


,Username,rawTweet,hashtags,mentions,fromUser
0,La_ReVoluZzion,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",,"usembassyguate, 48CantonesToto, USAIDGuate, UE...",XelaNewsGt
1,XelaNewsGt,#URGENTE Lo que los medios #faferos no informa...,"URGENTE, faferos, BernardoArévalo, NebajQuiché...",,
2,M24095273,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,,"IvanDuque, BArevalodeLeon",
3,carlosalbesc,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,,"IvanDuque, BArevalodeLeon",
4,Brenda_AGN,El presidente @BArevalodeLeon y la vicepreside...,,"BArevalodeLeon, KarinHerreraVP, AGN_noticias, ...",


Username    0
rawTweet    0
hashtags    0
mentions    0
fromUser    0
dtype: int64

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)

STOPWORDS_ES = set(stopwords.words('spanish'))

STOPWORDS_ES.update({
    "rt","vía","via"
})

len(STOPWORDS_ES)


316

En este caso, como también se quiere hacer un análisis de emociones, los emojis se van a guardar en una columna separada, pero se van a eliminar del tweet para la columna de "clean".

In [15]:
import re
import emoji

# URLs
URL_RE = re.compile(r"https?://\S+|www\.\S+", flags=re.IGNORECASE)

# Extrae la lista de emojis en el texto (mantiene repeticiones)
def list_emojis(s: str):
    return [m["emoji"] for m in emoji.emoji_list(s or "")]

def remove_emojis(s: str) -> str:
    return emoji.replace_emoji(s or "", replace=" ")

def clean_tweet_text(s: str, stopset: set) -> str:
    if not isinstance(s, str):
        return ""

    # 1) minúsculas
    s = s.lower()

    # 2) quitar urls
    s = URL_RE.sub(" ", s)

    # 3) quitar emojis
    s = remove_emojis(s)

    # 4) quitar @ y # pero conservar el resto
    s = s.replace("@", " ").replace("#", " ")

    # 5) quitar signos de puntuación/símbolos (mantener letras a-z + acentos y espacios)
    s = re.sub(r"[^0-9a-záéíóúüñ\s]", " ", s)

    # 6) normalizar espacios
    s = re.sub(r"\s+", " ", s).strip()

    # 7) tokenización simple + filtros por dígitos y stopwords
    tokens = []
    for tok in s.split():
        if any(ch.isdigit() for ch in tok):  # descartar tokens con dígitos
            continue
        if tok in stopset:
            continue
        tokens.append(tok)

    return " ".join(tokens)


In [17]:
# Extrae emojis y crea columnas auxiliares
df["emojis"] = df["rawTweet"].apply(list_emojis)                 # lista de emojis
df["emojis_str"] = df["emojis"].apply(lambda arr: " ".join(arr)) # útil para CSV más adelante
df["emoji_count"] = df["emojis"].apply(len)

# Crea clean_tweet SIN emojis
df["clean_tweet"] = df["rawTweet"].apply(lambda s: clean_tweet_text(s, STOPWORDS_ES))

df[["rawTweet","emojis_str","emoji_count","clean_tweet"]].head(20)


,rawTweet,emojis_str,emoji_count,clean_tweet
0,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",,0,confirmado compañeres impuesto usembassyguate ...
1,#URGENTE Lo que los medios #faferos no informa...,🇫🇷,1,urgente medios faferos informaron ayer acerca ...
2,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,,0,ivanduque barevalodeleon usaste pegasus espiar...
3,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,,0,ivanduque barevalodeleon entienden bien cuadra...
4,El presidente @BArevalodeLeon y la vicepreside...,,0,presidente barevalodeleon vicepresidenta karin...
5,@BArevalodeLeon El muy hijo de puta inyectó co...,,0,barevalodeleon hijo puta inyectó enfermedades ...
6,@EmisorasUnidas @BArevalodeLeon Y que de bueno...,,0,emisorasunidas barevalodeleon bueno trae guate...
7,@IvanDuque @BArevalodeLeon Productiva es que g...,,0,ivanduque barevalodeleon productiva genere pro...
8,"@KarinHerreraVP El presidente de la República,...",,0,karinherreravp presidente república barevalode...
9,"@mys_servicios @BArevalodeLeon No tampoco, ese...",,0,mys servicios barevalodeleon tampoco izmierda ...


In [ ]:
n0 = len(df)

# --- Duplicados EXACTOS (en todas las columnas) ---
cols_base = ["Username","rawTweet","hashtags","mentions","fromUser","clean_tweet","emojis_str","emoji_count"]
cols_exist = [c for c in cols_base if c in df.columns]
df = df.drop_duplicates(subset=cols_exist, keep="first").reset_index(drop=True)
n1 = len(df)
print(f"Exactos en {cols_exist}: {n0-n1} filas eliminadas. Total: {n1}")

# --- Duplicados por rawTweet NORMALIZADO ---
df["_raw_norm"] = (
    df["rawTweet"]
      .astype(str)
      .str.lower()
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)
df = df.drop_duplicates(subset=["_raw_norm"], keep="first").drop(columns=["_raw_norm"]).reset_index(drop=True)
n2 = len(df)
print(f"Por 'rawTweet' normalizado: {n1-n2} filas eliminadas. Total: {n2}")

# --- Duplicados por CONTENIDO LIMPIO (clean_tweet) ---
df = df.drop_duplicates(subset=["clean_tweet"], keep="first").reset_index(drop=True)
n3 = len(df)
print(f"Por 'clean_tweet': {n2-n3} filas eliminadas. Total final: {n3}")

print(f"\nResumen: eliminadas {n0-n3} de {n0} filas. Dataset final: {len(df)}")


Exactos en ['Username', 'rawTweet', 'hashtags', 'mentions', 'fromUser', 'clean_tweet', 'emojis_str', 'emoji_count']: 0 filas eliminadas. Total: 4688
Por 'rawTweet' normalizado: 0 filas eliminadas. Total: 4688
Por 'clean_tweet': 0 filas eliminadas. Total final: 4688

Resumen: eliminadas 0 de 4688 filas. Dataset final: 4688


Se hacen las relaciones según las menciones, retweets y replies.

In [ ]:
import re
import numpy as np
import pandas as pd

# Manejo de menciones
def parse_mentions_field(x: str):
    if not isinstance(x, str) or not x.strip():
        return []
    # separar por coma y limpiar
    parts = [p.strip().lstrip('@').lower() for p in x.split(',') if p.strip()]
    out = []
    for p in parts:
        if not p:
            continue
        # fallback por si vinieran espacios sin comas
        if ' ' in p:
            out.extend([q.strip().lstrip('@').lower() for q in p.split() if q.strip()])
        else:
            out.append(p)
    # filtra vacíos
    return [m for m in out if m]

RT_PATTERN    = re.compile(r'^\s*rt\s+@([a-z0-9_]{1,15})', re.IGNORECASE)
REPLY_PATTERN = re.compile(r'^\s*@([a-z0-9_]{1,15})',     re.IGNORECASE)

# Manejo de retweets
def get_retweet_target(text: str):
    if not isinstance(text, str): 
        return None
    m = RT_PATTERN.search(text)
    return m.group(1).lower() if m else None

# Manejo de replies
def get_reply_target(text: str):
    if not isinstance(text, str): 
        return None
    m = REPLY_PATTERN.search(text)
    return m.group(1).lower() if m else None

"""
Genera edges dirigidos:
    - mention: Username -> cada @mencion
    - quote:   Username -> fromUser (si no vacío)
    - retweet: Username -> @usuario si texto inicia con 'RT @usuario'
    - reply:   Username -> @usuario si texto inicia con '@usuario'
Devuelve DataFrame: [source, target, interaction, weight]
"""
def build_edges(df: pd.DataFrame) -> pd.DataFrame:
    edges = []

    for _, row in df.iterrows():
        src = (row.get("Username") or "").strip()
        if not src:
            continue
        src_l = src.lower()

        # Mentions
        for m in parse_mentions_field(row.get("mentions", "")):
            if m and m != src_l:
                edges.append((src_l, m, "mention"))

        # Quote
        q = (row.get("fromUser") or "").strip().lower()
        if q and q != src_l:
            edges.append((src_l, q, "quote"))

        # Retweet (heurístico)
        rt = get_retweet_target(row.get("rawTweet", ""))
        if rt and rt != src_l:
            edges.append((src_l, rt, "retweet"))

        # Reply (heurístico)
        rp = get_reply_target(row.get("rawTweet", ""))
        if rp and rp != src_l:
            edges.append((src_l, rp, "reply"))

    if not edges:
        return pd.DataFrame(columns=["source","target","interaction","weight"])

    edges_df = pd.DataFrame(edges, columns=["source","target","interaction"])
    edges_df["weight"] = 1

    # Agregar pesos por (source, target, interaction)
    edges_agg = (edges_df
                 .groupby(["source","target","interaction"], as_index=False)["weight"]
                 .sum()
                 .sort_values(["weight","source","target"], ascending=[False, True, True])
                 .reset_index(drop=True))
    return edges_agg

edges = build_edges(df)
print("Edges construidos:", edges.shape)
edges.head(10)


Edges construidos: (15701, 4)


,source,target,interaction,weight
0,elrevoltijogt,barevalodeleon,mention,42
1,benitoc67601310,barevalodeleon,mention,30
2,elrevoltijogt,bancadasemilla,mention,30
3,ialmgg,barevalodeleon,mention,21
4,diariodeca,wendiv_dca,mention,18
5,dupin07,barevalodeleon,mention,18
6,vvj_gt,barevalodeleon,mention,18
7,diariodeca,barevalodeleon,mention,16
8,diariodeca,mmaczdca,mention,16
9,anti_chairosgt,barevalodeleon,mention,15


Se hace una matriz de adyacencia para ver como se relacionan los usuarios

In [21]:
# Matriz de adyacencia
if not edges.empty:
    adj = (edges.groupby(["source","target"])["weight"]
                 .sum()
                 .unstack(fill_value=0)
                 .astype(int)
          )
    print("Adjacency shape:", adj.shape)
    adj.head(5)
else:
    adj = pd.DataFrame()


Adjacency shape: (2465, 1066)


Se pasa todo a csv limpios.

In [22]:
# Guardar limpio principal
CLEAN_CSV = "../data/clean_data.csv"
cols_to_keep = [c for c in [
    "Username","rawTweet","hashtags","mentions","fromUser",
    "clean_tweet","emojis_str","emoji_count"
] if c in df.columns]

df[cols_to_keep].to_csv(CLEAN_CSV, index=False, encoding="utf-8")
print(f"✓ Guardado dataset limpio en {CLEAN_CSV} (filas: {len(df)})")

# Guardar edges
EDGES_CSV = "../data/network_edges.csv"
edges.to_csv(EDGES_CSV, index=False, encoding="utf-8")
print(f"✓ Guardado aristas en {EDGES_CSV} (filas: {len(edges)})")

# Guardar matriz de adyacencia
if not adj.empty:
    ADJ_CSV = "../data/network_adjacency.csv"
    adj.to_csv(ADJ_CSV, encoding="utf-8")
    print(f"✓ Guardada matriz de adyacencia en {ADJ_CSV} (shape: {adj.shape})")


✓ Guardado dataset limpio en ../data/clean_data.csv (filas: 4688)
✓ Guardado aristas en ../data/network_edges.csv (filas: 15701)
✓ Guardada matriz de adyacencia en ../data/network_adjacency.csv (shape: (2465, 1066))
